In [0]:
!pip install -qU t5

In [0]:
PRETRAINED_MODEL_DIR = 'My Drive/MLLU/pretrained_models/base'

In [0]:
DATA_DIR = 'My Drive/MLLU/data'
MODEL_DIR = 'My Drive/MLLU/model'

In [0]:
import functools
import t5
import torch
import transformers
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# model = t5.models.HfPyTorchModel("t5-small", "/tmp/hft5/", device)
model = t5.models.hf_model.HfPyTorchModel("t5-base", MODEL_DIR, device)

In [0]:
import tensorflow_datasets as tfds
sample = True

def sst2_dataset_fn(split, shuffle_files=False):
  # train: 67349   validation: 872
  
  # We only have one file for each split.
  del shuffle_files

  ds = tfds.load(
    "glue/sst2",
    data_dir=DATA_DIR,
    # Download data locally for preprocessing to avoid using GCS space.
    download_and_prepare_kwargs={"download_dir": "./downloads"})
    #print("A few raw validation examples...")

  if split == 'validation':
    if sample:
      ds = ds['validation'].take(300)
    else:
      ds = ds['validation']

  elif split == "train":
    if sample:
      ds = ds['train'].take(600)
    else:
      ds = ds['train']
  # else:
  #   ds = ds['test']

  return ds


In [0]:
def label_preprocessor(ds):
  
  def to_inputs_and_targets(ex):
    return {
        "inputs": "sst2 sentence: " + ex["sentence"],
        "targets": "i" if ex["label"] == 0 else "c", 
        "idx": ex["idx"]
    }
  return ds.map(to_inputs_and_targets,
               num_parallel_calls=tf.data.experimental.AUTOTUNE)



In [0]:

t5.data.TaskRegistry.remove("sst2")
t5.data.TaskRegistry.add(
    "sst2",
    # A TfdsTask takes in a TFDS name instead of a tf.data.Dataset function.
    dataset_fn=sst2_dataset_fn,
    splits=["train", "validation"],
    
    sentencepiece_model_path=t5.data.DEFAULT_SPM_PATH,
    text_preprocessor=[label_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.accuracy]
)

# Load and print a few examples.
import tensorflow.compat.v1 as tf


In [0]:
mrpc_task = t5.data.TaskRegistry.get("sst2")
ds = mrpc_task.get_dataset(split='train', sequence_length={"inputs": 128, "targets": 4})
print("A few preprocessed validation examples...")
max_ = 0
count = 0
for ex in tfds.as_numpy(ds.take(20)):
    print(ex)
    count += 1
    max_=max(max_,len(ex["targets"]))
print(max_)
print(count)

In [0]:
# Evaluate the pre-trained checkpoint, before further fine-tuning
model.eval(
    "sst2",
    sequence_length={"inputs": 128, "targets": 4},
    batch_size=128,
)

In [0]:
model.train(
    mixture_or_task_name="sst2",
    steps=2000,
    save_steps=200,
    sequence_length={"inputs": 128, "targets": 4},
    split="train",
    batch_size=32,
    optimizer=functools.partial(transformers.AdamW, lr=1e-4),
)

In [0]:
# Evaluate after fine-tuning
model.eval(
    "sst2",
    checkpoint_steps="all",
    sequence_length={"inputs": 128, "targets": 4},
    batch_size=128,
)